In [3]:
import pandas as pd

#Import get_weather function that queries weather API
from get_weather import get_weather

#Use this if get weather is in a different directory
#  import sys
# # sys.path is a list of absolute path strings
# sys.path.append('/path/to/application/app/folder')

# import file

ModuleNotFoundError: No module named 'get_weather'

In [5]:
#Read airportcoordinates text file which contains airport codes and coordinates in a dictionary
import ast

def readtxt():
  file = open("airportcoordinates.txt", "r")

  contents = file.read()
  dictionary = ast.literal_eval(contents)

  file.close()
  
  return dictionary

In [22]:
#Input data into get_weather function
start_dates = ['2018-03-01','2018-04-02','2018-05-02','2018-06-02','2018-07-02']  # N0TE: OFFSET BY 1 DAY EACH START DATE AS QUERY IS INCLUSIVE
end_dates = ['2018-04-01','2018-05-01','2018-06-01','2018-07-01','2018-08-01']
coords = readtxt()




In [ ]:
#Iterate through coord dictionary and call get_weather to query API
for key,value in coords.items():
  get_weather(start_dates,end_dates,key,value)

In [4]:
#Load returned CSVs from the get_weather function
dat = pd.read_csv('test_final.csv')
dat = dat[dat['airport_code'] != 'airport_code']
dat.head(3)

NameError: name 'pd' is not defined

In [36]:
#Check what each hourly value looks like
dat['hourly'][1]

"[{'time': '24', 'tempC': '12', 'tempF': '54', 'windspeedMiles': '11', 'windspeedKmph': '18', 'winddirDegree': '282', 'winddir16Point': 'WNW', 'weatherCode': '116', 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'}], 'weatherDesc': [{'value': 'Partly cloudy'}], 'precipMM': '0.0', 'precipInches': '0.0', 'humidity': '34', 'visibility': '10', 'visibilityMiles': '6', 'pressure': '1007', 'pressureInches': '30', 'cloudcover': '9', 'HeatIndexC': '7', 'HeatIndexF': '45', 'DewPointC': '-8', 'DewPointF': '18', 'WindChillC': '5', 'WindChillF': '42', 'WindGustMiles': '16', 'WindGustKmph': '26', 'FeelsLikeC': '5', 'FeelsLikeF': '42', 'uvIndex': '3'}]"

In [37]:
# Function to remove string literal that is enclosing our hourly column
import ast

def parse_hourly(row):
  return ast.literal_eval(row)

In [38]:
#Remove the string literal that is enclosing our hourly column
dat['hourly'] = dat.apply(lambda row: parse_hourly(row['hourly']), axis=1)

In [39]:
#Check what each hourly value looks like now (no string)
dat['hourly'][1]

[{'DewPointC': '-8',
  'DewPointF': '18',
  'FeelsLikeC': '5',
  'FeelsLikeF': '42',
  'HeatIndexC': '7',
  'HeatIndexF': '45',
  'WindChillC': '5',
  'WindChillF': '42',
  'WindGustKmph': '26',
  'WindGustMiles': '16',
  'cloudcover': '9',
  'humidity': '34',
  'precipInches': '0.0',
  'precipMM': '0.0',
  'pressure': '1007',
  'pressureInches': '30',
  'tempC': '12',
  'tempF': '54',
  'time': '24',
  'uvIndex': '3',
  'visibility': '10',
  'visibilityMiles': '6',
  'weatherCode': '116',
  'weatherDesc': [{'value': 'Partly cloudy'}],
  'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'}],
  'winddir16Point': 'WNW',
  'winddirDegree': '282',
  'windspeedKmph': '18',
  'windspeedMiles': '11'}]

In [40]:
# Explode hourly column dictionary into its own dataframe where dictionary keys are columns
dat_hourly = dat.hourly.apply(pd.Series)

In [41]:
dat_hourly.head(3)

,0
0,"{'time': '24', 'tempC': '11', 'tempF': '52', '..."
1,"{'time': '24', 'tempC': '12', 'tempF': '54', '..."
2,"{'time': '24', 'tempC': '7', 'tempF': '45', 'w..."


In [42]:
#Normalise weatherDesc dictionary into seperate dataframe
dat_hourly = pd.json_normalize(dat_hourly[0])

In [43]:
dat_hourly.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,24,11,52,2,3,211,SSW,113,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Sunny'}],0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3
1,24,12,54,11,18,282,WNW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3
2,24,7,45,17,27,323,NW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3


In [44]:
#Get rid of list bracket format in weatherDesc column
testdf = dat_hourly.explode('weatherDesc')

In [45]:
testdf.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,24,11,52,2,3,211,SSW,113,[{'value': 'http://cdn.worldweatheronline.com/...,{'value': 'Sunny'},0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3
1,24,12,54,11,18,282,WNW,116,[{'value': 'http://cdn.worldweatheronline.com/...,{'value': 'Partly cloudy'},0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3
2,24,7,45,17,27,323,NW,116,[{'value': 'http://cdn.worldweatheronline.com/...,{'value': 'Partly cloudy'},0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3


In [46]:
#Normalise weatherDesc dictionary into seperate dataframe
weatherDescription_df = pd.json_normalize(testdf['weatherDesc'])

In [47]:
weatherDescription_df.head(3)

,value
0,Sunny
1,Partly cloudy
2,Partly cloudy


In [48]:
#Merge weatherDescription_df with dat_hourly on index
dat_hourly_merged = dat_hourly.merge(weatherDescription_df, how = 'outer', left_index =True, right_index=True )

In [49]:
dat_hourly_merged.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex,value
0,24,11,52,2,3,211,SSW,113,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Sunny'}],0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,24,12,54,11,18,282,WNW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,24,7,45,17,27,323,NW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy


In [50]:
#Drop weatherDesc column and any other useless columns
dat_hourly_merged = dat_hourly_merged.drop(columns=['weatherIconUrl','weatherDesc'])
dat_hourly_merged.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex,value
0,24,11,52,2,3,211,SSW,113,0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,24,12,54,11,18,282,WNW,116,0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,24,7,45,17,27,323,NW,116,0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy


In [51]:
#Merge dat_hourly with original dat dataframe read from csv
dat_new = dat.merge(dat_hourly_merged, how ='outer', left_index=True, right_index=True)

In [52]:
dat_new.head(3)

,date,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex_x,hourly,airport_code,astronomy,maxtempC,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex_y,value
0,2018-03-01,52,4,39,8,47,0.0,8.7,3,"[{'time': '24', 'tempC': '11', 'tempF': '52', ...",ORD,"[{'sunrise': '06:44 AM', 'sunset': '05:58 PM',...",11,24,11,52,2,3,211,SSW,113,0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,2018-03-02,54,3,38,8,47,0.0,8.7,3,"[{'time': '24', 'tempC': '12', 'tempF': '54', ...",ORD,"[{'sunrise': '06:42 AM', 'sunset': '05:59 PM',...",12,24,12,54,11,18,282,WNW,116,0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,2018-03-03,45,4,39,6,42,0.0,11.6,3,"[{'time': '24', 'tempC': '7', 'tempF': '45', '...",ORD,"[{'sunrise': '06:41 AM', 'sunset': '06:00 PM',...",7,24,7,45,17,27,323,NW,116,0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy


In [53]:
#Drop useless columns in dat_new
dat_new.drop(columns=['hourly','astronomy'])

,date,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex_x,airport_code,maxtempC,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex_y,value
0,2018-03-01,52,4,39,8,47,0.0,8.7,3,ORD,11,24,11,52,2,3,211,SSW,113,0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,2018-03-02,54,3,38,8,47,0.0,8.7,3,ORD,12,24,12,54,11,18,282,WNW,116,0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,2018-03-03,45,4,39,6,42,0.0,11.6,3,ORD,7,24,7,45,17,27,323,NW,116,0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy
3,2018-03-04,42,-1,31,3,38,0.0,11.6,1,ORD,6,24,6,42,3,5,185,S,119,0.0,0.0,31,10,6,1022,31,23,3,37,-13,9,2,35,4,7,2,35,1,Cloudy
4,2018-03-05,48,1,33,5,41,0.0,11.6,3,ORD,9,24,9,48,4,6,192,SSW,113,0.0,0.0,33,10,6,1020,31,13,4,40,-11,13,3,38,6,10,3,38,3,Sunny
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,2018-07-28,69,7,44,13,55,0.0,14.5,4,MSP,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2940,2018-07-29,67,10,50,13,56,0.0,14.5,3,MSP,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2941,2018-07-30,71,11,52,16,60,0.0,14.5,4,MSP,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2942,2018-07-31,63,9,47,11,52,0.0,9.3,3,MSP,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
dat_new.value.value_counts

<bound method IndexOpsMixin.value_counts of 0               Sunny
1       Partly cloudy
2       Partly cloudy
3              Cloudy
4               Sunny
            ...      
2939              NaN
2940              NaN
2941              NaN
2942              NaN
2943              NaN
Name: value, Length: 2944, dtype: object>

In [60]:
# Check NaN values
# missing data
total = dat_new.isnull().sum().sort_values(ascending=False)
percent = (dat_new.isnull().sum()/dat_new.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
#missing_data

missing_data

,Total,Percent
value,18,0.006114
winddir16Point,18,0.006114
windspeedKmph,18,0.006114
windspeedMiles,18,0.006114
tempF,18,0.006114
tempC,18,0.006114
time,18,0.006114
maxtempC,18,0.006114
astronomy,18,0.006114
airport_code,18,0.006114
